# 회귀

## 학생성적 예측 데이터

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv")


display(x_train.head())
display(y_train.head())

,StudentID,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,1714,GP,F,18,U,GT3,T,4,3,other,...,no,4,3,3,1,1,3,0,14,13
1,1254,GP,F,17,U,GT3,T,4,3,health,...,yes,4,4,3,1,3,4,0,13,15
2,1639,GP,F,16,R,GT3,T,4,4,health,...,no,2,4,4,2,3,4,6,10,11
3,1118,GP,M,16,U,GT3,T,4,4,services,...,no,5,3,3,1,3,5,0,15,13
4,1499,GP,M,19,U,GT3,T,3,2,services,...,yes,4,5,4,1,1,4,0,5,0


,StudentID,G3
0,1714,14
1,1254,15
2,1639,11
3,1118,13
4,1499,0


In [5]:
# print(x_train.isna().sum().sum())  ## 결측값 없음
print(x_train.info())
print(x_train.nunique())
# object columns 더미화 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   StudentID   678 non-null    int64 
 1   school      678 non-null    object
 2   sex         678 non-null    object
 3   age         678 non-null    int64 
 4   address     678 non-null    object
 5   famsize     678 non-null    object
 6   Pstatus     678 non-null    object
 7   Medu        678 non-null    int64 
 8   Fedu        678 non-null    int64 
 9   Mjob        678 non-null    object
 10  Fjob        678 non-null    object
 11  reason      678 non-null    object
 12  guardian    678 non-null    object
 13  traveltime  678 non-null    int64 
 14  studytime   678 non-null    int64 
 15  failures    678 non-null    int64 
 16  schoolsup   678 non-null    object
 17  famsup      678 non-null    object
 18  paid        678 non-null    object
 19  activities  678 non-null    object
 20  nursery   

In [8]:
## drop
x_train_drop = x_train.drop(columns='StudentID')
x_test_drop = x_test.drop(columns='StudentID')
## dummies
x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]
y = y_train.G3

## split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

xt, xv, yt, yv = train_test_split(x_train_dummies, y, test_size=0.3, random_state=0)
rf = RandomForestRegressor(random_state=0)
rf.fit(xt, yt)

## predict
yv_pred = rf.predict(xv)

## 모델 평가
# mse : mean_squared_error
# mae : mean_absolute_error 
# mape : mean_absolute_percentage_error
# rmse : root_mean_squerd_error -> 패키지 없음 np.sqrt(mean_squared_error)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
print('== val ==')
print('mean_squared_error :', mean_squared_error(yv, yv_pred))
print('mean_absolute_error :', mean_absolute_error(yv, yv_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yv, yv_pred))
print('rmse :', np.sqrt(mean_squared_error(yv, yv_pred)))
print('r2_score :', r2_score(yv, yv_pred))

pred = rf.predict(x_test_dummies)
pd.DataFrame({'StudentID':x_test.StudentID, 'G3':pred}).to_csv('result_regression.csv', index=False)

== val ==
mean_squared_error : 1.1072933743191722
mean_absolute_error : 0.7524191176470587
mean_absolute_percentage_error : 437776375542926.2
rmse : 1.0522800835895223
r2_score : 0.9286134000516384


## 중고차 가격 예측 데이터

In [9]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [17]:
print(x_train[['brand','model','transmission','fuelType']].nunique(), '\n',x_test[['brand','model','transmission','fuelType']].nunique())
## drop model

brand            9
model           90
transmission     4
fuelType         5
dtype: int64 
 brand            9
model           89
transmission     4
fuelType         5
dtype: int64


In [18]:
pd.concat([x_train_drop, x_test_drop])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,GP,F,18,U,GT3,T,4,3,other,other,...,no,4,3,3,1,1,3,0,14,13
1,GP,F,17,U,GT3,T,4,3,health,other,...,yes,4,4,3,1,3,4,0,13,15
2,GP,F,16,R,GT3,T,4,4,health,teacher,...,no,2,4,4,2,3,4,6,10,11
3,GP,M,16,U,GT3,T,4,4,services,services,...,no,5,3,3,1,3,5,0,15,13
4,GP,M,19,U,GT3,T,3,2,services,at_home,...,yes,4,5,4,1,1,4,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,GP,F,15,R,LE3,T,3,1,other,other,...,no,4,4,2,2,3,3,6,15,15
362,GP,M,17,U,GT3,T,3,3,services,services,...,yes,4,3,4,2,3,4,12,12,12
363,GP,F,16,U,GT3,T,2,3,services,teacher,...,no,2,3,1,1,1,3,2,16,16
364,MS,F,16,U,GT3,T,3,1,other,other,...,no,3,1,3,1,3,1,0,8,6


In [14]:
print(x_test.info(), x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         2672 non-null   int64  
 1   brand         2672 non-null   object 
 2   model         2672 non-null   object 
 3   year          2672 non-null   int64  
 4   transmission  2672 non-null   object 
 5   mileage       2672 non-null   int64  
 6   fuelType      2672 non-null   object 
 7   tax           2672 non-null   float64
 8   mpg           2672 non-null   float64
 9   engineSize    2672 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 208.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         4960 non-null   int64  
 1   brand         4960 non-null   object 
 2   model         4960 non-null  

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

x_train_drop = x_train.drop(columns = 'carID')
x_test_drop = x_test.drop(columns = 'carID')
y = y_train['price']

# train에만 있고 test에는 없는 model들이 있기에 합쳐서 원핫 인코딩을 진행
combined = pd.concat([x_train_drop, x_test_drop])
combined_encoded = pd.get_dummies(combined)
train_encoded = combined_encoded[:len(x_train_drop)]
test_encoded = combined_encoded[len(x_train_drop):]

xt , xv , yt, yv = train_test_split(train_encoded , y, test_size=0.3, random_state=0)
rf = RandomForestRegressor(random_state =42)
rf.fit(xt, yt)

yv_pred = rf.predict(xv)

print('== val ==')
print('mean_squared_error :', mean_squared_error(yv, yv_pred))
print('mean_absolute_error :', mean_absolute_error(yv, yv_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yv, yv_pred))
print('rmse :', np.sqrt(mean_squared_error(yv, yv_pred)))
print('r2_score :', r2_score(yv, yv_pred))

pred = rf.predict(test_encoded)
pd.DataFrame({'carID':x_test.carID, 'price':pred}).to_csv('result_regression.csv', index=False)

== val ==
mean_squared_error : 12861197.338756274
mean_absolute_error : 2077.9584928207432
mean_absolute_percentage_error : 0.10592654437084767
rmse : 3586.251153887061
r2_score : 0.9538282515270621


## 의료 비용 예측 데이터

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,age,sex,bmi,children,smoker,region
0,2,35,female,35.860,2,no,southeast
1,3,28,female,23.845,2,no,northwest
2,4,23,female,32.780,2,yes,southeast
3,6,52,female,25.300,2,yes,southeast
4,7,63,male,39.800,3,no,southwest


,ID,charges
0,2,5836.52040
1,3,4719.73655
2,4,36021.01120
3,6,24667.41900
4,7,15170.06900


In [9]:
# print(x_train.info(), x_test.info())
## 결측치 없음
## object: sex, smoker, region

# print(x_train[['sex', 'smoker', 'region']].value_counts(), x_test[['sex', 'smoker', 'region']].value_counts())
## 카테고리 종류는 train, test 동일

## drop
x_traind = x_train.drop(columns=['ID', 'region'])
x_testd = x_test.drop(columns=['ID', 'region'])

## dummies (원핫인코딩)
x_traindd = pd.get_dummies(x_traind)
x_testdd = pd.get_dummies(x_testd)
x_testdd = x_testdd[x_traindd.columns]

## modeling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

y = y_train['charges']
X_train, X_val, Y_train, Y_val = train_test_split(x_traindd, y, test_size=0.3, random_state=0)

rfr = RandomForestRegressor(random_state=0)
rfr.fit(X_train, Y_train)

## predict
import numpy as np
t_pred = rfr.predict(X_train)
v_pred = rfr.predict(X_val)

print('== train ==')
print('mean_squared_error :', mean_squared_error(Y_train, t_pred))
print('mean_absolute_error :', mean_absolute_error(Y_train, t_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_train, t_pred))
print('r2_score :', r2_score(Y_train, t_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_train, t_pred)))
print('\n')
print('== val ==')
print('mean_squared_error :', mean_squared_error(Y_val, v_pred))
print('mean_absolute_error :', mean_absolute_error(Y_val, v_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_val, v_pred))
print('r2_score :', r2_score(Y_val, v_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_val, v_pred)))

pred = rfr.predict(x_testdd)
pd.DataFrame({'ID':x_test.ID, 'charges':pred}).to_csv('result2.csv', index=False)

== train ==
mean_squared_error : 3708048.4323675293
mean_absolute_error : 1061.6644624660883
mean_absolute_percentage_error : 0.12728892181277815
r2_score : 0.9740723464660108
rmse : 1925.6293600710208


== val ==
mean_squared_error : 24663350.40541378
mean_absolute_error : 2683.147715914329
mean_absolute_percentage_error : 0.31622380600998634
r2_score : 0.8484699638959567
rmse : 4966.220938038679


## 킹카운티 주거지 가격예측문제 데이터

In [10]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2,8651400730,20150428T000000,3,1.00,840,5525,1.0,0,0,...,6,840,0,1969,0,98042,47.3607,-122.085,920,5330
1,3,3163600130,20150317T000000,3,1.00,1250,8000,1.0,0,0,...,7,1250,0,1956,0,98146,47.5065,-122.337,1040,6973
2,4,5045700330,20140725T000000,4,2.50,2200,6400,2.0,0,0,...,8,2200,0,2010,0,98059,47.4856,-122.156,2600,5870
3,5,1036100130,20140808T000000,3,2.50,1980,39932,2.0,0,0,...,8,1980,0,1994,0,98011,47.7433,-122.196,2610,12769
4,6,7696630080,20140506T000000,3,1.75,1690,7735,1.0,0,0,...,7,1060,630,1976,0,98001,47.3324,-122.280,1580,7503


,ID,price
0,2,191000.0
1,3,234900.0
2,4,460000.0
3,5,442000.0
4,6,197000.0


In [15]:
# print(x_train.info(), x_test.info())
## 결측치 없음
## object : date

x_traind = x_train.drop(columns=['ID','id','date'])
x_testd = x_test.drop(columns=['ID','id','date'])

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

y = y_train['price']
X_train, X_val, Y_train, Y_val = train_test_split(x_traind, y, test_size=0.3, random_state=0)

rfr = RandomForestRegressor(random_state=0)
rfr.fit(X_train, Y_train)

t_pred = rfr.predict(X_train)
v_pred = rfr.predict(X_val)

print('== train ==')
print('mean_squared_error :', mean_squared_error(Y_train, t_pred))
print('mean_absolute_error :', mean_absolute_error(Y_train, t_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_train, t_pred))
print('r2_score :', r2_score(Y_train, t_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_train, t_pred)))
print('\n')
print('== val ==')
print('mean_squared_error :', mean_squared_error(Y_val, v_pred))
print('mean_absolute_error :', mean_absolute_error(Y_val, v_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_val, v_pred))
print('r2_score :', r2_score(Y_val, v_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_val, v_pred)))

pred = rfr.predict(x_testd)
pd.DataFrame({'ID':x_test.ID, 'price':pred}).to_csv('result2.csv', index=False)

== train ==
mean_squared_error : 2487615272.681608
mean_absolute_error : 26603.10415911246
mean_absolute_percentage_error : 0.050450257703648964
r2_score : 0.9813206132206322
rmse : 49875.9989642474


== val ==
mean_squared_error : 21101743236.637653
mean_absolute_error : 73973.20272321372
mean_absolute_percentage_error : 0.1345228673332978
r2_score : 0.8644402768892553
rmse : 145264.39080737458


## 대학원 입학가능성 데이터

In [24]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,0,67,327,114,3,3.0,3.0,9.02,0
1,1,112,321,109,4,4.0,4.0,8.68,1
2,2,495,301,99,3,2.5,2.0,8.45,1
3,3,356,317,106,2,2.0,3.5,8.12,0
4,4,250,321,111,3,3.5,4.0,8.83,1


,ID,Chance of Admit
0,0,0.61
1,1,0.69
2,2,0.68
3,3,0.73
4,4,0.77


In [27]:
# print(x_train.info(), x_test.info())
## object 없음
## 결측치 없음

drops = ['ID', 'Serial No.']
x_traind = x_train.drop(columns=drops)
x_testd = x_test.drop(columns=drops)
y = y_train['Chance of Admit']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

X_train, X_val, Y_train, Y_val = train_test_split(x_traind, y, test_size=0.3, random_state=0)
rfr = RandomForestRegressor(random_state=0)
rfr.fit(X_train, Y_train)

t_pred = rfr.predict(X_train)
v_pred = rfr.predict(X_val)

print('== train ==')
print('mean_squared_error :', mean_squared_error(Y_train, t_pred))
print('mean_absolute_error :', mean_absolute_error(Y_train, t_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_train, t_pred))
print('r2_score :', r2_score(Y_train, t_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_train, t_pred)))
print('\n')
print('== val ==')
print('mean_squared_error :', mean_squared_error(Y_val, v_pred))
print('mean_absolute_error :', mean_absolute_error(Y_val, v_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(Y_val, v_pred))
print('r2_score :', r2_score(Y_val, v_pred))
print('rmse :', np.sqrt(mean_squared_error(Y_val, v_pred)))

pred = rfr.predict(x_testd)
pd.DataFrame({'ID':x_test.ID, 'Chance of Admit': pred}).to_csv('result2.csv', index=False)

== train ==
mean_squared_error : 0.0005760361071428567
mean_absolute_error : 0.016950357142857142
mean_absolute_percentage_error : 0.02713519797086307
r2_score : 0.9711765476419025
rmse : 0.02400075222035461


== val ==
mean_squared_error : 0.004815572749999997
mean_absolute_error : 0.0499041666666667
mean_absolute_percentage_error : 0.07951759057390534
r2_score : 0.7636315222745069
rmse : 0.06939432793824


## 레드 와인 퀄리티 예측 데이터

In [39]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,1,10.6,0.44,0.68,4.1,0.114,6.0,24.0,0.99700,3.06,0.66,13.4
1,2,7.0,0.60,0.30,4.5,0.068,20.0,110.0,0.99914,3.30,1.17,10.2
2,3,8.0,0.43,0.36,2.3,0.075,10.0,48.0,0.99760,3.34,0.46,9.4
3,4,7.9,0.53,0.24,2.0,0.072,15.0,105.0,0.99600,3.27,0.54,9.4
4,5,8.0,0.45,0.23,2.2,0.094,16.0,29.0,0.99620,3.21,0.49,10.2


,ID,quality
0,1,6
1,2,5
2,3,5
3,4,6
4,5,6


In [40]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    320 non-null    int64  
 1   fixed acidity         320 non-null    float64
 2   volatile acidity      320 non-null    float64
 3   citric acid           320 non-null    float64
 4   residual sugar        320 non-null    float64
 5   chlorides             320 non-null    float64
 6   free sulfur dioxide   320 non-null    float64
 7   total sulfur dioxide  320 non-null    float64
 8   density               320 non-null    float64
 9   pH                    320 non-null    float64
 10  sulphates             320 non-null    float64
 11  alcohol               320 non-null    float64
dtypes: float64(11), int64(1)
memory usage: 30.1 KB


In [47]:
# print(x_train.info())
# 결측치 없음, object type 없음, 다중 분류로 풀기

x_traind = x_train.drop(columns='ID')
x_testd = x_test.drop(columns='ID')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

y = y_train['quality']
X_train, X_val, Y_train, Y_val = train_test_split(x_traind, y, test_size=0.3, random_state=0, stratify=y)
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, Y_train)

t_pred = rfc.predict(X_train)
v_pred = rfc.predict(X_val)
t_proba = rfc.predict_proba(X_train)
v_proba = rfc.predict_proba(X_val)

print('== train ==')
print('accuracy_score :', accuracy_score(Y_train, t_pred))
print('recall_score :', recall_score(Y_train, t_pred, average='weighted'))
print('precision_score :', precision_score(Y_train, t_pred, average='weighted'))
print('f1_score :', f1_score(Y_train, t_pred, average='weighted'))
print('\n')
print('== val ==')
print('accuracy_score :', accuracy_score(Y_val, v_pred))
print('recall_score :', recall_score(Y_val, v_pred, average='weighted'))
print('precision_score :', precision_score(Y_val, v_pred, average='weighted'))
print('f1_score :', f1_score(Y_val, v_pred, average='weighted'))

pred = rfc.predict(x_testd)
proba = rfc.predict_proba(x_testd)[:,1]
pd.DataFrame({'ID':x_test.ID, 'quality':pred}).to_csv('result.csv', index=False)
pd.DataFrame({'ID':x_test.ID, 'quality-proba':proba}).to_csv('result2.csv', index=False)

== train ==
accuracy_score : 1.0
recall_score : 1.0
precision_score : 1.0
f1_score : 1.0


== val ==
accuracy_score : 0.6979166666666666
recall_score : 0.6979166666666666
precision_score : 0.6700047348484848
f1_score : 0.678485402525177


C:\Users\na\anaconda3\envs\torch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 현대 차량 가격 분류문제 데이터

In [78]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_test.csv")


display(x_train.head())
display(y_train.head())

,ID,model,year,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,0,I30,2019,Manual,21,Petrol,150,34.0,2.0
1,1,Santa Fe,2018,Semi-Auto,10500,Diesel,145,39.8,2.2
2,2,Tucson,2017,Manual,29968,Diesel,30,61.7,1.7
3,3,Kona,2018,Manual,27317,Petrol,145,52.3,1.0
4,4,Tucson,2018,Semi-Auto,31459,Diesel,145,57.7,1.7


,ID,price
0,0,23995
1,1,28490
2,2,13251
3,3,14990
4,4,17591


In [80]:
# print(x_train.info(), x_test.info())
# print(y_train.info())
## 결측치 없음, object type : model, transmission, fuelType

# print(x_train[['model', 'transmission', 'fuelType']].nunique())
# print(x_test[['model', 'transmission', 'fuelType']].nunique())
## train, test의 카테고리에 차이가 있음. 합쳐서 원핫인코딩

## drop
x_traind = x_train.drop(columns='ID')
x_testd = x_test.drop(columns='ID')

## 원핫인코딩
x_all = pd.concat([x_traind, x_testd])
all_encoding = pd.get_dummies(x_all)
x_traindd = all_encoding[:len(x_traind)]
x_testdd = all_encoding[len(x_traind):]

y = y_train['price']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

xt, xv, yt, yv = train_test_split(x_traindd, y, test_size=0.3, random_state=0)
rf = RandomForestRegressor(random_state=0)
rf.fit(xt, yt)

tpred = rf.predict(xt)
vpred = rf.predict(xv)

print('== train ==')
print('mean_squared_error :', mean_squared_error(yt, tpred))
print('mean_absolute_error :', mean_absolute_error(yt, tpred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yt, tpred))
print('r2_score :', r2_score(yt, tpred))
print('rmse :', np.sqrt(mean_squared_error(yt, tpred)))
print('\n')
print('== val ==')
print('mean_squared_error :', mean_squared_error(yv, vpred))
print('mean_absolute_error :', mean_absolute_error(yv, vpred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yv, vpred))
print('r2_score :', r2_score(yv, vpred))
print('rmse :', np.sqrt(mean_squared_error(yv, vpred)))

pred = rf.predict(x_testdd)
pd.DataFrame({'ID':x_test.ID, 'price':pred}).to_csv('result_regression.csv', index=False)

== train ==
mean_squared_error : 240807.4892302312
mean_absolute_error : 323.5837810515468
mean_absolute_percentage_error : 0.026498161833110153
r2_score : 0.9929627548141994
rmse : 490.72139675199736


== val ==
mean_squared_error : 1604523.1838402068
mean_absolute_error : 865.4025334418508
mean_absolute_percentage_error : 0.07664031885052124
r2_score : 0.9545423226264993
rmse : 1266.6977476257732
